In [1]:
import tensorflow as tf
import numpy as np
import tensorboard as tb

# 変数の表示(show variable)
変数に名前を付けることで、可視化したときにわかりやすくすることができる。  
やり方は変数の宣言の中に`name='hoge'`と入れれば良い。

# 名前空間(name space)
可視化したときにわかりやすくしたり、C++等のように変数名等が被らないようにする効果がある。 
やり方は`with tf.name_scope('hoge'):`と書き、内部に処理を書く。

## 簡単な例(simple example)
以下の式を計算してみる。
\begin{eqnarray}
x &=& 1\\
y &=& 2\\
a &=& x \cdot y\\
b &=& x+y\\
z &=& \frac{x \cdot y}{x+y}
\end{eqnarray}

In [2]:
tf.reset_default_graph()

x = tf.constant(1, name='x-input')
y = tf.constant(2, name='y-input')

with tf.Session() as sess:
    sess.run([tf.multiply(x, y, name='a'), tf.add(x, y, name='b')])
    sess.run(tf.div(tf.multiply(x, y, name='a'), tf.add(x, y, name='b'), name='z-output'))
    
    tf.summary.FileWriter('./log/', sess.graph)

tb.show_graph(tf.get_default_graph().as_graph_def())

### 少し複雑な例(little complicated example)
以下の計算を実行してみる。
\begin{eqnarray}
x_{data} &=& \begin{pmatrix}
             x_{11}   & x_{12}   & \ldots & x_{15} \\
             x_{21}   & x_{22}   & \ldots & x_{25} \\
             \vdots   & \vdots   & \ddots & \vdots \\
             x_{1001} & x_{1002} & \ldots & x_{1005} 
             \end{pmatrix}\\
W &=& \begin{pmatrix}
      W_{11}   & W_{12} & W_{13} \\
      W_{21}   & W_{22} & W_{23} \\
      \vdots   & \vdots & \vdots \\
      W_{51} & W_{52}   & W_{53} 
      \end{pmatrix}\\
b &=& \begin{pmatrix}
      0 & 0 & 0 \\
      \end{pmatrix}\\
y &=& x_{data} \cdot w + b
\end{eqnarray}

In [ ]:
tf.reset_default_graph()

x_data = tf.random_uniform([100, 5], name='x_data')
with tf.Session() as sess:
    W = tf.Variable(tf.random_uniform([5, 3], -1.0, 1.0), name='W')
    b = tf.Variable(tf.zeros([3]), name='b')
    y = tf.add(tf.matmul(x_data, W), b, name='y')
    
    
    tf.summary.FileWriter('./log/', sess.graph)

tb.show_graph(tf.get_default_graph().as_graph_def())

## サンプル機械学習モデル(Sample machine learning model)
以下のようなモデルを定義する。
\begin{eqnarray}
y &=& Ax + b
\end{eqnarray}  
xは入力データ、yは出力データとする。
学習の結果、以下のようになるモデルを組む。
\begin{eqnarray}\\
I &=&
\begin{pmatrix}
1&0&0\\
0&1&0\\
0&0&1
\end{pmatrix}\\
A &=& 2I\\
b &=&
\begin{pmatrix}
0\\
0\\
0
\end{pmatrix}\\\\
\begin{pmatrix}
y_{1}\\
y_{2}\\
y_{3}
\end{pmatrix}
&=& 
\begin{pmatrix}
2&0&0\\
0&2&0\\
0&0&2
\end{pmatrix}
\begin{pmatrix}
x_{1}\\
x_{2}\\
x_{3}
\end{pmatrix}\\\\
\end{eqnarray}
このモデル実現のために、トレーニングデータを100個用意して学習する。

In [8]:
tf.reset_default_graph()

sess = tf.InteractiveSession()

#パラメータ
A = tf.Variable(tf.zeros(shape=[3,3]), name='A')
b = tf.Variable(tf.zeros(shape=[3,1]), name='b')

#入力
x = tf.placeholder(dtype=tf.float32, shape=[3,1], name='x')
#出力
y = tf.placeholder(dtype=tf.float32, shape=[3,1], name='y')

#モデル定義
y_ =tf.add(tf.matmul(A, x), b) 

#誤差の定義
with tf.name_scope('loss') :
    loss = tf.reduce_mean(tf.square(y_ - y))
    tf.summary.scalar('loss', loss)

#トレーニングアルゴリズム
with tf.name_scope('train') :
    learning_rate = 1.0e-3
    train = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

with tf.name_scope('summary') :
    writer = tf.summary.FileWriter('/tmp/tensorflow/test-tensorboard/logs/basic', sess.graph)
    tf.summary.scalar('A', tf.reshape(tf.matrix_determinant(A), []))
    tf.summary.scalar('b', tf.reshape(tf.norm(b), []))
    tf.summary.scalar('loss', loss)
    merged = tf.summary.merge_all()

sess.run(tf.global_variables_initializer())

# トレーニングデータを生成
samples_count = 100
x_input = np.ndarray(shape=[samples_count,3,1])
y_input = np.ndarray(shape=[samples_count,3,1])
for i in range(samples_count):
    x_input[i][0] = np.random.rand()
    x_input[i][1]= np.random.rand()
    x_input[i][2] = np.random.rand()

    for j in range(3):
        y_input[i][j] = x_input[i][j]* 2

# 学習開始
summary = None
total_epochs = 1000
print('Start learning...')
for _ in range(total_epochs):
    for i in range(samples_count):
        __, summary = sess.run([train, merged], feed_dict={x: x_input[i], y: y_input[i]})

    # 進捗10%毎に途中経過をコンソール出力、TensorBoardに監視対象変数の値を記録
    if _% (total_epochs/10) == 0:
        curr_A, curr_b, curr_loss= sess.run([A, b, loss], feed_dict={x: x_input[_% samples_count], y: y_input[_% samples_count]})
        
        writer.add_summary(summary, _)

# 学習結果の確認
# x = (1,2,3) を入力すると (2,4,6)に近いベクトルが出力されるはずだが、実際にどうなるか確認。
x_test= np.array([1,2,3]).reshape([3,1])
y_test = np.array([2,4,6]).reshape([3,1])
estimated_y, final_loss = sess.run([y_, loss], feed_dict={x:x_test, y: y_test})
print('y')
print(y_input[_% samples_count])
print('A')
print(curr_A)
print('x')
print(x_input[_% samples_count])
print('b')
print(curr_b)
print('loss')
print(curr_loss)
print('Input x')
print(x_test)
print('Computed y')
print(estimated_y)
print('loss')
print(final_loss)

summary = tf.summary.merge_all()
tf.summary.FileWriter('./log/', sess.graph)
tb.show_graph(tf.get_default_graph().as_graph_def())

Start learning...
y
[[ 1.42945904]
 [ 0.77547324]
 [ 1.79357794]]
A
[[  1.97285318e+00  -1.02871228e-02  -7.22695549e-05]
 [ -1.02693886e-02   1.98234928e+00  -3.36122396e-03]
 [  3.29014583e-05  -3.21915979e-03   1.95461404e+00]]
x
[[ 0.71472952]
 [ 0.38773662]
 [ 0.89678897]]
b
[[ 0.01972039]
 [ 0.01725943]
 [ 0.02490908]]
loss
0.000150706
Input x
[[1]
 [2]
 [3]]
Computed y
[[ 1.97887814]
 [ 3.97320366]
 [ 5.91923189]]
loss
0.00256255


In [9]:
!tensorboard --logdir=/tmp/tensorflow/test-tensorboard/logs/basic

Starting TensorBoard b'47' at http://0.0.0.0:6006
(Press CTRL+C to quit)
^C
